FUTURE TO DO: LOCALISE TEMPERATURE

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import copy
class daisy():
    def __init__(self,color, total, cov):
        #Standard constants
        self.D = 0.3
        self.ALB = 0.75 if color == 'white' else 0.25
        self.INSUL = 0.2
        self.temp = 0 #TEMPORARY
        self.growth = 0 #TEMPORARY
        self.cov = cov #the first cov is set by us
        
        self.d_growth = 0
        self.d_death = 0
        
        self.total = total #this is used to update coverage because it is now dependent on more factors
        self.color = -1 if color == 'white' else 1 #This is for data plotting
        
        
        self.potential_coords = [] #This is for Conway's implementation
        self.death_row = [] #this is for life_death
        self.coords = []
        
    def d_temp(self, lumin, INSOL, SB, p_alb, p_temp4):
        self.temp = ((self.INSUL * lumin) * \
        (INSOL/SB) * (p_alb - self.ALB) + p_temp4) ** (1/4)
        
    def growth_r(self):
        self.growth = 1 - 0.003265 * ((295.65 - self.temp)**2)
        #print("growth rate is:" + str(self.growth))
    
    def get_growth(self, g_cov, dt):
        #This gives me an ABSOLUTE value
        self.d_growth =  int(self.cov * self.total * g_cov * self.growth * dt)
        #print("growth rate is:" + str(self.d_growth))
        
    def get_death(self, dt):
        #This gives me an ABSOLUTE value
#         print(str(self.cov) + " "+str(self.total)+" "+str(self.D)+" "+str(dt))
        self.d_death = int(self.cov * self.total * self.D * dt)
#         print("calculated: " + str(self.d_death))
    
    def update_cov(self):
        self.cov =  len(self.coords) / self.total

In [ ]:
class world():

    def __init__(self, total= 10000, b_cov = 0.1, w_cov = 0.1, t = 100, dt = 1, static = True):
        #Default constants
        self.INSOL = 917
        self.SB = 5.670367*1e-8
        self.ALB = 0.5
        self.p_alb = 0 #TEMPORARY
        self.temp4 = 22.5+273.15 #TEMPORARY
        self.lumin = [1 for i in range(int(t/dt))] if static==True else list(np.linspace(0.5,1.6,int(t/dt)))
        #list(np.linspace(1.0,1.6,int(t/dt))) OR #list(np.linspace(0.5,1.6,int(t/dt)))

        self.total = total
        self.t = t
        self.dt = dt
        self.cov = 0 #TEMPORARY
        self.min_number = 10 #arbitary value
        
        #Data collection
        self.t_lst = list(np.linspace(0,t,int(t/dt)))
        self.wcov_lst = []
        self.bcov_lst = []
        self.temp_lst = []
        
        self.bdaisy_coll = daisy('black', total, b_cov)
        self.wdaisy_coll = daisy('white', total, w_cov)
        
        #I will have two boards, a daisy board and a temperature board
        grid = int(total**0.5)
        self.d_board = [[0 for i in range(grid)]  for j in range(grid)] #Filling of board will not be done in the initialization step.
        self.t_board = [[0 for i in range(grid)]  for j in range(grid)] #For temperature
        self.n_board = [[[0,0] for i in range(grid)] for j in range(grid)] #For neighbors, can only make with list comprehension due to depth
        
        self.board_hash = {} #This will store all necessary information that I would like in my final output
        
        #Creation of Coordinates
        d_coords = np.random.choice(grid, (int(total*b_cov)+int(total*w_cov),2))
        while True:
            #Checking for duplicates
            d_coords = set(map(tuple,d_coords))
            
            #Making the code less cancerous to read
            no_unique = len(d_coords)
            no_expect = int(total*b_cov)+int(total*w_cov)
            
            if (no_unique == no_expect): #Break when I have no duplicates
                #Converting back to lists and np.arrays
                d_coords = list(map(np.array, d_coords)) 
                break 
                break
            else:
                to_add = np.random.choice(grid, (no_expect-no_unique,2)) #create missing elements
                d_coords = list(map(list,d_coords)) #converting set back to list
                d_coords.extend(to_add) #extending my list to prepare for next operation
                
        #Out of sheer laziness, I will assign the first half to b and the second to white
        d_coords = list(map(list,d_coords))
        self.bdaisy_coll.coords.extend(d_coords[:int(total*b_cov)])
        self.wdaisy_coll.coords.extend(d_coords[int(total*b_cov):])
    
    def ground_cov(self):
        self.cov = 1 - self.bdaisy_coll.cov - self.wdaisy_coll.cov #Establish my ground
    
    def alb(self):
        self.p_alb = (self.bdaisy_coll.cov * self.bdaisy_coll.ALB) + \
        (self.wdaisy_coll.cov * self.wdaisy_coll.ALB) + \
        (self.cov * self.ALB)
    
    def p_temp(self, lumin):
        self.temp4 = lumin * ((self.INSOL/self.SB) * (1 - self.p_alb))
    
    def update_board(self):
        """This method updates both board states with the necessary calculations.
        The daisy board will simply be a show of three basic colors,
        1. Green = Ground, represented by 0
        2. White = White daisies, represented by -1
        3. Black = Black daisies, represented by 1
        The temperature board will simply reflect the temperature values based on Ti
        
        I will use a lazy implementation for neighbor coordinates
        Since we already have collated all their positional coordinates, we can do it the lazy way
        We will just check all 4 surrounding coordinates in black and white daisies and then update the position with a list
        """
        grid = int(self.total**(0.5))
        for row in range(grid):
            for col in range(grid):
                #Updating my d_board and my t_board
                if [row,col] in self.wdaisy_coll.coords: #white daisies
                    self.d_board[row][col] = -1
                    #self.t_board[row][col] = self.wdaisy_coll.temp #this is for without gradient
                elif [row,col] in self.bdaisy_coll.coords: #black daisies
                    self.d_board[row][col] = 1
                    #self.t_board[row][col] = self.bdaisy_coll.temp #this is for without gradient
                else:
                    self.d_board[row][col] = 0
                    #self.t_board[row][col] = self.temp4 **(1/4) #this is for without gradient
                wdaisy = 0
                bdaisy = 0
                if [row-1,col] in self.wdaisy_coll.coords:
                    wdaisy += 1
                if [row,col-1] in self.wdaisy_coll.coords:
                    wdaisy += 1
                if [row,col+1] in self.wdaisy_coll.coords:
                    wdaisy += 1
                if [row+1,col] in self.wdaisy_coll.coords:
                    wdaisy += 1
                #Perform the exact same operations for black daisies
                if [row-1,col] in self.bdaisy_coll.coords:
                    bdaisy += 1
                if [row,col-1] in self.bdaisy_coll.coords:
                    bdaisy += 1
                if [row,col+1] in self.bdaisy_coll.coords:
                    bdaisy += 1
                if [row+1,col] in self.bdaisy_coll.coords:
                    bdaisy += 1
                    
                #Now, I have a true count of my neighbors, if I were to apply a diffused gradient, I can use my neighbors
                if self.d_board[row][col] == 0:
                    temp_val = self.temp4 ** (1/4)
                elif self.d_board[row][col] == -1:
                    temp_val = self.wdaisy_coll.temp
                elif self.d_board[row][col] == 1:
                    temp_val = self.bdaisy_coll.temp
                    
                temp_val = (temp_val + (wdaisy*self.wdaisy_coll.temp) + (bdaisy*self.bdaisy_coll.temp))/(1 + wdaisy + bdaisy)
                self.t_board[row][col] = temp_val-273.15

                #Checking for my edges, if either is at 2, we set them to 4 instead. This implementation will make sense considering life_death
                if (row == 0 and col == 0) or (row == 0 and col == grid-1) or (row == grid-1 and col == 0 ) or (row == grid-1 and col == grid-1):
                    if wdaisy == 2:
                        wdaisy = 4
                    if bdaisy == 2:
                        bdaisy = 4
                #Checking the edges, either one of them needs to be either 0 or grid-1 for it to be an edge:
                elif row == 0 or row == grid-1 or col == 0 or col == grid-1:
                    if wdaisy == 3:
                        wdaisy = 4
                    if bdaisy == 3:
                        bdaisy = 4            
                    
                self.n_board[row][col] = [wdaisy,bdaisy]
                
    
    def life_death(self):
        """Now that our grid is updated, wee can use that grid to perform our calculations.
        Here are the following rules to Conways game of life v1.1.
        I will do some basic explanation here:
        We have 3 types of cases in a 2d plane on Conways game of life.
        
        Case 1: Four corners (0,0), (0,n), (n,0), (n,n)
        Maximum surround plants is 2. 
        If there are 2 plants, the grid of interest DIES(0)
        If there is 1 plant, then if the growth rate is positive, we will run a probability that it grows!
        
        Case 2: All four sides (0, 0->n), (1->n, 0), (n, 1->n), (1->n-1, n)
        Maximum surround plants is 3
        If there are 3 plants, the grid of interest DIES(0)
        If there are 2 plants, 
            if 2 plants growth rate are different, we will run a probability that the higher one grows
            if that plant does not grow, we give the 2nd plant a chance to grow
            if their growth rate is equal, we will RNG
        If there is 1 plant, then if the growth rate is positive, we will run a probability that it grows!
        
        At this point of time, we can boil everything down to three cases
        1. Death
        2. Potential growth (one plant)
        3. Comparison of growth
        
        Case 3: Everywhere else in the grid
        Maximum surround plants is 4
        If there are 4 plants, death case
        If there are 3 plants, we will calculate based on raw value (ie neighboring quantity instead of growth)
        If there are 2 plants, comparison of growth
        If there is 1 plant, potential growth
        
        I will collapse all of this into one function called checker
        """
        def checker(wdaisy_no, bdaisy_no):
            #Case 0: DIE. This is also the reason for changing neighbors to 4. It is for standardization purposes
            if wdaisy_no == 4 or bdaisy_no == 4:
                return -100
            #Case 1: I have more white daisies than black daisies
            if wdaisy_no > bdaisy_no :
                #I will return white daisy if growth is positive
                if self.wdaisy_coll.d_growth > 0:
                    return self.wdaisy_coll.color 
                #I will return black daisy if black daisy growth rate is positive
                if (self.bdaisy_coll.d_growth > 0):
                    return self.bdaisy_coll.color 
            
            #Case 2: I have more black daisies than white daisies
            if bdaisy_no > wdaisy_no :
                #I will return black daisy if growth is positive
                if self.bdaisy_coll.d_growth > 0:
                    return self.bdaisy_coll.color 
                #I will return white daisy if white daisy growth rate is positive and it is actually present
                if (self.wdaisy_coll.d_growth > 0):
                    return self.wdaisy_coll.color 
            
            #Case 3: I have an equal number of black and white daisies
            if self.wdaisy_coll.growth > self.bdaisy_coll.growth > 0:
                return self.wdaisy_coll.color
            if self.bdaisy_coll.growth > self.wdaisy_coll.growth > 0:
                return self.bdaisy_coll.color
            
            #If it does not satisfy any condition
            return 0
        
        grid = int(self.total**(0.5))
        #We have made a neighbors board, lets use that
        for row in range(grid):
            for col in range(grid):
                #As long as I do not have 0 neighbors each, I can throw them into the helper function
                if ((self.n_board[row][col])[0] != 0) or ((self.n_board[row][col])[1] != 0):
                    to_add = checker(((self.n_board[row][col])[0]), ((self.n_board[row][col])[1]))
                    #print(to_add)
                    #append to white daisy coordinates if it is not yet present
                    if to_add == -1 and self.d_board[row][col] != -1 and [row,col] not in self.wdaisy_coll.potential_coords:
                        self.wdaisy_coll.potential_coords.extend([[row,col],])
                    #append to black daisy coordinates if it is not yet present
                    if to_add == 1 and self.d_board[row][col] != 1 and [row,col] not in self.bdaisy_coll.potential_coords:
                        self.bdaisy_coll.potential_coords.extend([[row,col],])
                    #return value 0, set it to cull depending on which daisy is present
                    if to_add == -100:
                        if [row,col] in self.wdaisy_coll.coords: #This is a white daisy asking to be culled
                            self.wdaisy_coll.death_row.extend([[row,col],])
                        if [row,col] in self.bdaisy_coll.coords: #This is a white daisy asking to be culled
                            self.bdaisy_coll.death_row.extend([[row,col],])

        
    def run(self):
        """My intialization only creates daisies and an empty board.
        Thus, I will need to update everything at t0 as well"""
        for i in range(int(self.t/self.dt)):
            self.ground_cov()
            self.alb()
            
            
            self.p_temp(self.lumin[i])
            self.p_temp(self.lumin[i])
            self.bdaisy_coll.d_temp(self.lumin[i], self.INSOL, self.SB, self.p_alb, self.temp4)
            self.wdaisy_coll.d_temp(self.lumin[i], self.INSOL, self.SB, self.p_alb, self.temp4)
            self.bdaisy_coll.growth_r()
            self.wdaisy_coll.growth_r()
            
            #these are in absolute values
            self.wdaisy_coll.get_growth(self.cov, self.dt)
            self.bdaisy_coll.get_growth(self.cov, self.dt)
            self.wdaisy_coll.get_death(self.dt)
            self.bdaisy_coll.get_death(self.dt)
            
#             print("white: " + str(self.wdaisy_coll.total)+ " " + str(self.wdaisy_coll.cov) +" "+ str(self.wdaisy_coll.D) +" "+ str(self.dt))
#             print(self.wdaisy_coll.d_death)
#             print("black: " + str(self.bdaisy_coll.total) +" " + str(self.bdaisy_coll.cov) +" " + str(self.bdaisy_coll.D) +" " + str(self.dt))
#             print(self.bdaisy_coll.d_death)
            
            #By this point of the first iteration, I have now updated all the potential variables that I need
            #First update the board
            self.update_board()
            
            #Now we run growth and death calculations
            self.life_death() #Only need to run this if my daisies are growing
            
            """At this point in time, we should have two following important things before proceeding
            1. White and black daisies potential areas of growth
            2. List of potentially dead daisies
            We are going to kill the existing daisies before adding the new daisies
            1. If my expected total death is more than what I have for dead daisies they will all die.
               Whatever that is left will be randomly sampled from the daisy population
            2. If my expected total death is less than what I have I will sample from the dead_lst
            Similarly for growing new daisies
            1. If my expected total growth is more than what I have for the potential growth, they will all grow
            2. However, since the daisies are facing spatial constraints, we shall not spawn daisies like miracles.
            """
            #Let the culling begin
            #print(self.n_board)
            def helper_kill(daisy_coll):
                #print("kill is run for " + str(daisy_coll.color))
                daisy_kill = daisy_coll.d_death
                add_kill = False
                
                if len(daisy_coll.death_row) < daisy_kill: 
                    add_kill = daisy_kill - len(daisy_coll.death_row)
                    
#                 if len(daisy_coll.death_row) != 0:
#                     if len(daisy_coll.death_row) > daisy_kill:
#                         np.random.shuffle(daisy_coll.death_row)
#                         daisy_coll.death_row = daisy_coll.death_row[:daisy_kill]
                                
                for coord in daisy_coll.death_row:
#                     print("removed coord")
#                     print(coord)
                    daisy_coll.coords.remove(coord)
            
                if add_kill:
                    daisy_coll.coords = daisy_coll.coords[add_kill:]
            
#             print(self.n_board)
#             print("white before death length is: " + str(len(self.wdaisy_coll.coords)))
#             print("white death: " + str(self.wdaisy_coll.d_death))
#             test_a = len(self.wdaisy_coll.coords)
#             print(len(self.wdaisy_coll.coords))
#             print(self.wdaisy_coll.death_row)
#             print("white potential")
#             print(self.wdaisy_coll.potential_coords)
            
#             print("black death: "+ str(self.bdaisy_coll.d_death))
#             print(len(self.bdaisy_coll.coords))
#             print(self.bdaisy_coll.death_row)
#             print("black potential")
#             print(self.bdaisy_coll.potential_coords)
            
            #Calling both functions only when they are more than my minimum number
            if len(self.wdaisy_coll.coords) > self.min_number:
                helper_kill(self.wdaisy_coll)
            if len(self.bdaisy_coll.coords) > self.min_number:
                helper_kill(self.bdaisy_coll)
                
#             print(self.wdaisy_coll.coords)
#             print("white after death length is: " + str(len(self.wdaisy_coll.coords)))
#             test_b = len(self.wdaisy_coll.coords)
#             if test_a == test_b:
#                 print(self.wdaisy_coll.coords)
            
            #Let the growing begin
            def helper_grow(daisy_coll):
                daisy_grow = daisy_coll.d_growth
                
                if len(daisy_coll.potential_coords) > daisy_grow:
                    np.random.shuffle(daisy_coll.potential_coords)
                    daisy_coll.potential_coords = daisy_coll.potential_coords[:daisy_grow]
                
                daisy_coll.coords.extend(daisy_coll.potential_coords)
            
            #Calling both functions
            helper_grow(self.wdaisy_coll)
            helper_grow(self.bdaisy_coll)
            
            #Now I need to reupdate the coverage values of white and black daisies
            self.wdaisy_coll.update_cov()
            self.bdaisy_coll.update_cov()
            
#             print("white daisy coverage is now: " + str(self.wdaisy_coll.cov))
#             print("black daisy coverage is now: " + str(self.bdaisy_coll.cov))
            
            #Once I am done using the lists, I need to clear them
            self.wdaisy_coll.death_row = []
            self.bdaisy_coll.death_row = []
            self.wdaisy_coll.potential_coords = []
            self.bdaisy_coll.potential_coords = []
            
            self.wcov_lst.append(self.wdaisy_coll.cov)
            self.bcov_lst.append(self.bdaisy_coll.cov)
            self.temp_lst.append((self.temp4 ** (1/4)-273.15))
            self.board_hash[self.t_lst[i]] = [copy.deepcopy(self.d_board), copy.deepcopy(self.t_board)]
        return self.board_hash, self.wcov_lst, self.bcov_lst, self.temp_lst